# Data Cleaning

This notebook contains the code used for cleaning the "HK_AirBnB_Data_raw.csv" data, exporting it into the "cleaned_HK_AirBnB_Data.csv".

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read the raw data
df = pd.read_csv(r"../dat/HK_AirBnB_Data_raw.csv")

In [3]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [4]:
len(df)

7029

In [5]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
count,7.029000e+03,7.029000e+03,0.0,7029.000000,7029.000000,6552.000000,7029.000000,7029.000000,3541.000000,7029.000000,7029.000000,7029.000000,0.0
mean,3.776090e+17,1.594925e+08,NaN,22.296967,114.163259,891.273657,20.400626,14.146251,0.888749,98.409020,216.688149,3.278133,NaN
std,4.403533e+17,1.778368e+08,NaN,0.034115,0.046004,2851.987331,42.007246,41.788835,1.146428,137.396629,146.416534,8.472504,NaN
min,1.789100e+04,6.303800e+04,NaN,22.191250,113.856340,1.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000,NaN
25%,2.755237e+07,1.771335e+07,NaN,22.280540,114.155570,217.000000,1.000000,0.000000,0.140000,3.000000,69.000000,0.000000,NaN
50%,4.385338e+07,8.087394e+07,NaN,22.291929,114.171470,499.000000,7.000000,1.000000,0.450000,19.000000,263.000000,0.000000,NaN
75%,8.461219e+17,2.448217e+08,NaN,22.310550,114.180670,920.250000,29.000000,7.000000,1.150000,136.000000,364.000000,2.000000,NaN
max,1.049754e+18,5.510371e+08,NaN,22.547197,114.362470,150000.000000,1125.000000,511.000000,8.910000,398.000000,365.000000,119.000000,NaN


In [6]:
# drop columns since it has no/few values
df.drop(['neighbourhood_group', 'license', 'last_review', 'reviews_per_month'], axis=1, inplace=True)

In [7]:
# count of each room type
df['room_type'].value_counts()

Private room       3875
Entire home/apt    2747
Shared room         339
Hotel room           68
Name: room_type, dtype: int64

In [8]:
# one hot encode 'room_type' column
df = pd.get_dummies(df, columns=['room_type'])

In [9]:
# rename 'neighbourhood' into 'district'
df.rename(columns = {'neighbourhood': 'true_district'}, inplace = True)

In [10]:
# count all rows with atleast one missing value
df.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
true_district                       0
latitude                            0
longitude                           0
price                             477
minimum_nights                      0
number_of_reviews                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
room_type_Entire home/apt           0
room_type_Hotel room                0
room_type_Private room              0
room_type_Shared room               0
dtype: int64

In [11]:
def split_name(text):
    elements = re.split(r'\s*·\s*★?|\s*·\s*', text)
    return elements

def extract_last_element(series):
    return series.apply(lambda x: x[-1] if isinstance(x, list) and len(x) > 0 else None)

def extract_second_last_element(series):
    return series.apply(lambda x: x[-2] if isinstance(x, list) and len(x) > 1 else None)

def extract_third_last_element(series):
    return series.apply(lambda x: x[-3] if isinstance(x, list) and len(x) > 2 else None)

def extract_first_element(series):
    return series.apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

def extract_second_element(series):
    return series.apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)

In [12]:
import re
df['name_temp'] = df['name'].apply(split_name)

df['num_baths'] = extract_last_element(df['name_temp'])
df['num_beds'] = extract_second_last_element(df['name_temp'])
df['num_bedrooms'] = extract_third_last_element(df['name_temp'])
df['name'] = extract_first_element(df['name_temp'])
df['rating'] = extract_second_element(df['name_temp'])


In [13]:
# list of allowed bedroom counts
allowed_bedroom_counts = [
    "1 bedroom",
    "Studio",
    "2 bedrooms",
    "3 bedrooms",
    "4 bedrooms",
    "5 bedrooms",
    "6 bedrooms",
    "9 bedrooms",
    "7 bedrooms",
    "10 bedrooms",
    "8 bedrooms",
    "11 bedrooms"
]

# filter the df to keep only rows with 'num_bedrooms' in the allowed list
df = df[df['num_bedrooms'].isin(allowed_bedroom_counts)]

In [14]:
df['bedroom_count'] = df['num_bedrooms'].replace('Studio', '0').str.extract('(\d+)').astype(int)

allowed_bedroom_counts = list(range(0, 12)) 

df = df[df['bedroom_count'].isin(allowed_bedroom_counts)]

df.drop(['num_bedrooms'], axis=1, inplace=True)

In [15]:
df['bathroom_count'] = df['num_baths'].str.replace('(?i)half-bath', '0.5', regex=True)

df['bathroom_count'] = df['bathroom_count'].str.extract('(\d+\.?\d*)').astype(float)

allowed_bathroom_counts = np.arange(0, 12, 0.5)

df = df[df['bathroom_count'].isin(allowed_bathroom_counts)]

In [16]:
df.drop(['num_baths'], axis=1, inplace=True)

In [17]:
df['num_beds'].value_counts()

1 bed      4809
2 beds     1145
3 beds      408
4 beds      248
5 beds       81
6 beds       59
8 beds       46
10 beds      21
7 beds       17
9 beds       12
16 beds      11
11 beds       8
14 beds       1
15 beds       1
12 beds       1
13 beds       1
Name: num_beds, dtype: int64

In [18]:
df['bed_count'] = df['num_beds'].str.extract('(\d+)').astype(int)

allowed_bedroom_counts = list(range(0, 16))

df = df[df['bed_count'].isin(allowed_bedroom_counts)]

df.drop(['num_beds'], axis=1, inplace=True)

In [19]:
df.drop(['rating'], axis=1, inplace=True)

In [20]:
df.drop(['name_temp'], axis=1, inplace=True)

In [21]:
# one hot encode True_district
df = pd.get_dummies(df, columns=['true_district'])

In [22]:
# drop rows where price is missing
df = df.dropna(subset=['price'])

In [23]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,number_of_reviews_ltm,...,true_district_Sha Tin,true_district_Sham Shui Po,true_district_Southern,true_district_Tai Po,true_district_Tsuen Wan,true_district_Tuen Mun,true_district_Wan Chai,true_district_Wong Tai Sin,true_district_Yau Tsim Mong,true_district_Yuen Long
count,6.390000e+03,6.390000e+03,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,...,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000,6390.000000
mean,3.890393e+17,1.637249e+08,22.296732,114.163852,889.114867,20.206260,14.428482,102.899374,232.121283,3.471362,...,0.007981,0.024570,0.022535,0.004538,0.002973,0.004695,0.225352,0.003443,0.372770,0.007042
std,4.428073e+17,1.802190e+08,0.033145,0.045605,2868.029102,37.116194,42.620326,138.680579,139.234947,8.768070,...,0.088987,0.154822,0.148428,0.067219,0.054452,0.068363,0.417847,0.058580,0.483579,0.083629
min,7.257100e+04,6.303800e+04,22.191250,113.856340,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.743793e+07,1.825362e+07,22.280494,114.159905,217.000000,1.000000,0.000000,4.000000,90.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.404442e+07,9.447404e+07,22.292145,114.171620,500.000000,12.500000,0.000000,22.000000,271.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.520037e+17,2.524692e+08,22.310853,114.180737,925.000000,29.000000,7.000000,136.000000,364.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.049754e+18,5.510371e+08,22.547197,114.362470,150000.000000,365.000000,511.000000,398.000000,365.000000,119.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# export df
df.to_csv('cleaned_HK_AirBnB_Data.csv', index=False)

# Modeling

In [25]:
# pcr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from ISLP.models import ModelSpec as MS
import sklearn.linear_model as skl
import sklearn.model_selection as skm


df = pd.read_csv('cleaned_HK_AirBnB_Data.csv')

In [28]:
df.drop(['id', 'name', 'host_id', 'host_name'], axis=1, inplace=True)

In [29]:
design = MS(df.columns.drop('price')).fit(df)
X = np.asarray(design.fit_transform(df).drop('intercept', axis=1))
Y = np.array(df['price'])

In [30]:
print(design)

ModelSpec(terms=Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'bedroom_count', 'bathroom_count', 'bed_count',
       'true_district_Central & Western', 'true_district_Eas...
       'true_district_Kwai Tsing', 'true_district_Kwun Tong',
       'true_district_North', 'true_district_Sai Kung',
       'true_district_Sha Tin', 'true_district_Sham Shui Po',
       'true_district_Southern', 'true_district_Tai Po',
       'true_district_Tsuen Wan', 'true_district_Tuen Mun',
       'true_district_Wan Chai', 'true_district_Wong Tai Sin',
       'true_district_Yau Tsim Mong', 'true_district_Yuen Long'],
      dtype='object'))
